### Import Libraries

In [7]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [8]:
df18 = pd.read_csv('2018.csv')

In [9]:
df18['Year'] = '2018'

In [10]:
df17 = pd.read_csv('2017.csv')

In [11]:
df17['Year'] = '2017'

In [12]:
df16 = pd.read_csv('2016.csv')

In [13]:
df16['Year'] = '2016'

In [14]:
df15 = pd.read_csv("2015.csv")

In [15]:
df15['Year'] = '2015'

In [16]:
df15 = df15.rename(columns = {'Happiness Rank': 'Rank'} )
df16 = df16.rename(columns = {'Happiness Rank': 'Rank'} )
df17 = df17.rename(columns = {'Happiness.Rank': 'Rank', 'Happiness.Score': 'Happiness Score',  'Economy..GDP.per.Capita.':'Economy (GDP per Capita)', 'Health..Life.Expectancy':'Health (Life Expectancy)', 'Trust..Government.Corruption.':'Trust (Government Corruption)'})
df18 = df18.rename(columns = {'Score': 'Happiness Score','Social_Support':'Family', 'GDP_per_Capita':'Economy (GDP per Capita)', 'Freedom_To_Make_Life_Choices':'Freedom','Healthy_Life_Expectancy':'Health (Life Expectancy)', 'Perceptions_Of_Corruption':'Trust (Government Corruption)'})                   
                   

In [17]:
df = pd.concat([df15, df16,df17,df18])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
df.head()

,Country,Dystopia Residual,Dystopia.Residual,Economy (GDP per Capita),Family,Freedom,GDP_Per_Capita,Generosity,Happiness Score,Health (Life Expectancy),...,Lower Confidence Interval,Rank,Region,Residual,Standard Error,Trust (Government Corruption),Upper Confidence Interval,Whisker.high,Whisker.low,Year
0,Switzerland,2.51738,NaN,1.39651,1.34951,0.66557,NaN,0.29678,7.587,0.94143,...,NaN,1,Western Europe,NaN,0.03411,0.41978,NaN,NaN,NaN,2015
1,Iceland,2.70201,NaN,1.30232,1.40223,0.62877,NaN,0.43630,7.561,0.94784,...,NaN,2,Western Europe,NaN,0.04884,0.14145,NaN,NaN,NaN,2015
2,Denmark,2.49204,NaN,1.32548,1.36058,0.64938,NaN,0.34139,7.527,0.87464,...,NaN,3,Western Europe,NaN,0.03328,0.48357,NaN,NaN,NaN,2015
3,Norway,2.46531,NaN,1.45900,1.33095,0.66973,NaN,0.34699,7.522,0.88521,...,NaN,4,Western Europe,NaN,0.03880,0.36503,NaN,NaN,NaN,2015
4,Canada,2.45176,NaN,1.32629,1.32261,0.63297,NaN,0.45811,7.427,0.90563,...,NaN,5,North America,NaN,0.03553,0.32957,NaN,NaN,NaN,2015


### Rename variables

In [19]:
df['GDP'] = df[['Economy (GDP per Capita)', 'GDP_Per_Capita']].max(axis=1) # combine two columns together since data frames had different name

In [20]:
df['HealthLifeExpectancy'] = df[['Health (Life Expectancy)', 'Health..Life.Expectancy.']].max(axis=1)

In [21]:
df = df.rename(columns = {'Trust (Government Corruption)': 'Trust_GovCorruption', 'Happiness Score': 'HappinessScore'})

In [22]:
df = df[['Country', 'Rank', 'HappinessScore', 'Family', 
       'GDP', 'HealthLifeExpectancy', 'Freedom', 'Trust_GovCorruption', 'Generosity', 'Year']]

In [23]:
df.head()

,Country,Rank,HappinessScore,Family,GDP,HealthLifeExpectancy,Freedom,Trust_GovCorruption,Generosity,Year
0,Switzerland,1,7.587,1.34951,1.39651,0.94143,0.66557,0.41978,0.29678,2015
1,Iceland,2,7.561,1.40223,1.30232,0.94784,0.62877,0.14145,0.43630,2015
2,Denmark,3,7.527,1.36058,1.32548,0.87464,0.64938,0.48357,0.34139,2015
3,Norway,4,7.522,1.33095,1.45900,0.88521,0.66973,0.36503,0.34699,2015
4,Canada,5,7.427,1.32261,1.32629,0.90563,0.63297,0.32957,0.45811,2015


In [24]:
df.isna().sum()

Country                 0
Rank                    0
HappinessScore          0
Family                  0
GDP                     0
HealthLifeExpectancy    0
Freedom                 0
Trust_GovCorruption     1
Generosity              0
Year                    0
dtype: int64

In [25]:
# create new columns of continents
asia = ["Israel", "United Arab Emirates", "Singapore", "Thailand", "Taiwan Province of China",
                 "Qatar", "Saudi Arabia", "Kuwait", "Bahrain", "Malaysia", "Uzbekistan", "Japan",
                 "South Korea", "Turkmenistan", "Kazakhstan", "Turkey", "Hong Kong S.A.R., China", "Philippines",
                 "Jordan", "China", "Pakistan", "Indonesia", "Azerbaijan", "Lebanon", "Vietnam",
                 "Tajikistan", "Bhutan", "Kyrgyzstan", "Nepal", "Mongolia", "Palestinian Territories",
                 "Iran", "Bangladesh", "Myanmar", "Iraq", "Sri Lanka", "Armenia", "India", "Georgia",
                 "Cambodia", "Afghanistan", "Yemen", "Syria"]
europe = ["Norway", "Denmark", "Iceland", "Switzerland", "Finland",
                 "Netherlands", "Sweden", "Austria", "Ireland", "Germany",
                 "Belgium", "Luxembourg", "United Kingdom", "Czech Republic",
                 "Malta", "France", "Spain", "Slovakia", "Poland", "Italy",
                 "Russia", "Lithuania", "Latvia", "Moldova", "Romania",
                 "Slovenia", "North Cyprus", "Cyprus", "Estonia", "Belarus",
                 "Serbia", "Hungary", "Croatia", "Kosovo", "Montenegro",
                 "Greece", "Portugal", "Bosnia and Herzegovina", "Macedonia",
                 "Bulgaria", "Albania", "Ukraine"]
north_america = ["Canada", "Costa Rica", "United States", "Mexico",  
                 "Panama","Trinidad and Tobago", "El Salvador", "Belize", "Guatemala",
                 "Jamaica", "Nicaragua", "Dominican Republic", "Honduras",
                 "Haiti"]
south_america = ["Chile", "Brazil", "Argentina", "Uruguay",
                 "Colombia", "Ecuador", "Bolivia", "Peru",
                 "Paraguay", "Venezuela"]
australia = ["New Zealand", "Australia"]
d_asia = dict.fromkeys(asia, 'Asia')
d_europe = dict.fromkeys(europe, 'Europe')
d_north_america = dict.fromkeys(north_america, 'North America')
d_south_america = dict.fromkeys(south_america, 'South America')
d_australia = dict.fromkeys(australia, 'Australia')
continent_dict = {**d_asia, **d_europe, **d_north_america, **d_south_america, **d_australia}
df["continent"] = df["Country"].map(continent_dict)
df.continent.fillna("Africa", inplace=True)

### Check NAs

In [26]:
df.isna().sum()

Country                 0
Rank                    0
HappinessScore          0
Family                  0
GDP                     0
HealthLifeExpectancy    0
Freedom                 0
Trust_GovCorruption     1
Generosity              0
Year                    0
continent               0
dtype: int64

In [27]:
df['Trust_GovCorruption'].fillna(df['Trust_GovCorruption'].median(),inplace=True) # impute one missing data with median

In [28]:
df.shape

(626, 11)

In [29]:
df.to_csv('happiness.csv')

In [30]:
df.head()

,Country,Rank,HappinessScore,Family,GDP,HealthLifeExpectancy,Freedom,Trust_GovCorruption,Generosity,Year,continent
0,Switzerland,1,7.587,1.34951,1.39651,0.94143,0.66557,0.41978,0.29678,2015,Europe
1,Iceland,2,7.561,1.40223,1.30232,0.94784,0.62877,0.14145,0.43630,2015,Europe
2,Denmark,3,7.527,1.36058,1.32548,0.87464,0.64938,0.48357,0.34139,2015,Europe
3,Norway,4,7.522,1.33095,1.45900,0.88521,0.66973,0.36503,0.34699,2015,Europe
4,Canada,5,7.427,1.32261,1.32629,0.90563,0.63297,0.32957,0.45811,2015,North America


In [31]:
df = np.round(df, decimals=3) # round our data into 3 decimal places